In [71]:
from collections import Counter
from pathlib import Path

import json
import os
import pandas as pd

In [72]:
# 1. Lists all directories in the previous dir
# 2. Create a dict with the name of the models as keys
# 3. The values will be a new dict with the denomination of the "tasks_..." or "tasks" dir as key
# 4. The values of this last dict will be a new dict with the task (or split) as key so random_125 for example
# 5. The values of this last dict will be a new dict with the type of features (mmf e.g.) as key
# 6. The values of this last dict will be a new dict with the name of the metric as key
# 7. The values of this last dict will be the metric itself

In [73]:
# 1. Lists all directories in the previous dir
path_benchmarks = Path("..")
model_names = next(os.walk(path_benchmarks), (None, None, []))[1]
model_names.remove("results_stats")
model_names

['matten', 'modnet', 'mean_value', 'et']

In [74]:
# 2. Create a dict with the name of the models as keys
dict_results = {m: {} for m in model_names}
dict_results

{'matten': {}, 'modnet': {}, 'mean_value': {}, 'et': {}}

In [75]:
# 3. The values will be a new dict with the denomination of the "tasks_..." or "tasks" dir as key
for m in dict_results.keys():
    path_tasks = path_benchmarks / m
    for d in next(os.walk(path_tasks), (None, None, []))[1]:
        if "tasks" in d:
            dict_results[m].update({d: {}})

print(dict_results)

{'matten': {}, 'modnet': {'tasks': {}}, 'mean_value': {'tasks': {}}, 'et': {'tasks_dflt': {}, 'tasks_An': {}}}


In [76]:
# 4. The values of this last dict will be a new dict with the task (or split) as key so random_125 for example
for m in dict_results.keys():
    path_tasks = path_benchmarks / m
    for t in dict_results[m].keys():
        path_split = path_tasks / t
        for d in next(os.walk(path_split), (None, None, []))[1]:
            if "distribution" in d or "random" in d:
                dict_results[m][t].update({d: {}})

print(dict_results)

{'matten': {}, 'modnet': {'tasks': {'distribution_125': {}, 'distribution_250': {}, 'random_125': {}, 'random_250': {}}}, 'mean_value': {'tasks': {'random_250': {}}}, 'et': {'tasks_dflt': {'distribution_125': {}, 'distribution_250': {}, 'random_125': {}, 'random_250': {}}, 'tasks_An': {'distribution_125': {}, 'distribution_250': {}, 'random_125': {}, 'random_250': {}}}}


In [77]:
# 5. The values of this last dict will be a new dict with the type of features (mmf e.g.) as key
for m in dict_results.keys():
    path_tasks = path_benchmarks / m
    for t in dict_results[m].keys():
        path_split = path_tasks / t
        for s in dict_results[m][t].keys():
            path_feat = path_split / s

            for f in next(os.walk(path_feat), (None, None, []))[2]:
                if "json" not in f:
                    continue
                path_results = path_feat / f
                with open(path_results, 'r') as file:
                    results = json.load(file)
                dict_results[m][t][s].update({f.split(".json")[0]: results['metrics']})

# display(dict_results)


In [78]:
f = {}
stack = [(dict_results, '')]  # Stack holds tuples of (current_dict, current_key)

while stack:
    c, p = stack.pop()
    
    for k, v in c.items():
        new_key = f"{p}_{k}" if p else k
        
        if isinstance(v, dict):
            stack.append((v, new_key))  # Push the nested dictionary onto the stack
        else:
            f[new_key] = v  # Add to the flattened dictionary

# print(f)

In [79]:
dict_results_ravel = {}
for k, v in f.items():
    new_k, metric = k.replace("_tasks","").split("results_")
    if new_k not in dict_results_ravel.keys():
        dict_results_ravel[new_k] = {}
    dict_results_ravel[new_k][metric] = v

In [103]:
def df_style(val):
    return "color: red"
def df_style_bold(val):
    return "font-weight: bold"
import numpy as np
def highlight_max(s, props='color:red'):
    return np.where(s == np.nanmax(s.values), props, '')

df_results = pd.DataFrame.from_dict(dict_results_ravel, orient="index")

dict_df_results_splits = {}
for split in ["random_125", "random_250", "distribution_125", "distribution_250"]:
    indexes = []
    for i in df_results.index:
        if split in i:
            indexes.append(i)
    df_tmp = df_results.filter(indexes, axis=0)


    # and apply styling to it via the `subset` arg; first arg is styler function above
    idx_best = []
    idx_best.extend(df_tmp[['mae', 'rmse']].idxmin().tolist())
    idx_best.extend(df_tmp[['spearman', 'r2_score']].idxmax().tolist())
    df_tmp = df_tmp.style.applymap(df_style, subset=(idx_best, ["mae", "rmse", "spearman", "r2_score"]))

    idx_best_max, idx_best_max_count = Counter(idx_best).most_common(1)[0]
    if idx_best_max_count > 1:
        df_tmp = df_tmp.applymap(df_style_bold, subset=([idx_best_max], ["mae", "rmse", "spearman", "r2_score"]))

    dict_df_results_splits[split] = df_tmp

    print(split)
    display(dict_df_results_splits[split])


random_125


,mae,rmse,spearman,r2_score
et_An_random_125_pgnn_,8.647093,20.012587,0.886495,0.575846
et_An_random_125_mmf_,8.647552,19.931938,0.882402,0.579258
et_An_random_125_mmf_pgnn_,8.537435,19.984474,0.877677,0.577037
et_dflt_random_125_pgnn_,8.341509,19.831881,0.889002,0.583472
et_dflt_random_125_mmf_,8.192585,19.784299,0.892774,0.585468
et_dflt_random_125_mmf_pgnn_,8.307911,19.589507,0.894224,0.593591
modnet_random_125_pgnn_,8.262533,19.751812,0.901475,0.586828
modnet_random_125_mmf_,8.803178,23.035541,0.907785,0.438029
modnet_random_125_mmf_pgnn_,7.304253,19.423989,0.916129,0.600429


random_250


,mae,rmse,spearman,r2_score
et_An_random_250_pgnn_,8.912484,19.565746,0.855289,0.514493
et_An_random_250_mmf_,8.082588,18.117841,0.859724,0.583691
et_An_random_250_mmf_pgnn_,8.421179,18.395604,0.861294,0.570829
et_dflt_random_250_pgnn_,8.775236,19.223841,0.863953,0.531313
et_dflt_random_250_mmf_,7.545741,18.001918,0.888101,0.589002
et_dflt_random_250_mmf_pgnn_,8.358040,18.637842,0.881947,0.559451
mean_value_random_250_,16.953476,28.115598,nan,nan
modnet_random_250_pgnn_,7.761428,18.943342,0.891700,0.544891
modnet_random_250_mmf_,8.180140,19.811614,0.905217,0.502214
modnet_random_250_mmf_pgnn_,7.669056,19.340598,0.911464,0.525602


distribution_125


,mae,rmse,spearman,r2_score
et_An_distribution_125_pgnn_,7.918795,16.335552,0.791066,0.647445
et_An_distribution_125_mmf_,8.313777,16.655241,0.767476,0.633511
et_An_distribution_125_mmf_pgnn_,7.535542,15.169931,0.793524,0.695963
et_dflt_distribution_125_pgnn_,7.493740,15.093935,0.798667,0.699001
et_dflt_distribution_125_mmf_,6.839354,14.120979,0.813573,0.736556
et_dflt_distribution_125_mmf_pgnn_,7.202488,14.867961,0.815736,0.707947
modnet_distribution_125_pgnn_,6.542703,14.962140,0.855152,0.704235
modnet_distribution_125_mmf_,6.492044,13.945333,0.864332,0.743069
modnet_distribution_125_mmf_pgnn_,5.567543,12.074188,0.860172,0.807392


distribution_250


,mae,rmse,spearman,r2_score
et_An_distribution_250_pgnn_,6.823786,15.388477,0.831295,0.630368
et_An_distribution_250_mmf_,6.949693,15.245760,0.798448,0.637193
et_An_distribution_250_mmf_pgnn_,6.630075,15.324600,0.834100,0.633431
et_dflt_distribution_250_pgnn_,6.675432,15.839602,0.849602,0.608379
et_dflt_distribution_250_mmf_,6.144405,15.278746,0.837960,0.635621
et_dflt_distribution_250_mmf_pgnn_,6.189507,15.023034,0.844422,0.647716
modnet_distribution_250_pgnn_,5.817681,14.483580,0.869175,0.672562
modnet_distribution_250_mmf_,5.609491,14.656513,0.869088,0.664696
modnet_distribution_250_mmf_pgnn_,5.765881,15.279833,0.883553,0.635569
